Tuning hyperparameters is a time-consuming and computation-expensive task. To choose the right values, we can use heuristics or an extensive grid search. Grid search is a popular method for parameter tuning in machine learning.

In [34]:
# pip install hyperopt

import sys
import numpy as np

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.layers import Input
from keras.models import Model
from keras.applications.xception import Xception, preprocess_input

In [36]:
from Eve import Eve

opt_sgd = SGD(lr=1E-2, decay=1E-4, momentum=0.9, nesterov=True)
opt_adam = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1E-4)
opt_eve = Eve(lr=1E-4, decay=1E-4, beta_1=0.9, beta_2=0.999, beta_3=0.999, small_k=0.1, big_K=10, epsilon=1e-08)

In [37]:
nb_classes= 4
train_dir = 'data/training'
img_width, img_height = 150, 150
img_dim = (img_width, img_height, 3)

In [38]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
        width_shift_range=0.2,
        height_shift_range=0.2,
        fill_mode='nearest',
    horizontal_flip=True,
    validation_split=0.25)


test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    subset="training")

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    subset = "validation")

Found 108 images belonging to 4 classes.
Found 32 images belonging to 4 classes.


We can set the search space for different hyperparameters:

In [39]:
space = {
            'optimizer' : hp.choice('optimizer', ["opt_sgd", "opt_adam", "opt_eve"]), 'n_epochs' : 100,
        }


In [40]:
def get_callbacks(pars):
    callbacks =[EarlyStopping(monitor='val_acc', patience=5, verbose=2),
                ModelCheckpoint('grid_results/{}.h5'.format(pars['optimizer']), save_best_only=True),
             TensorBoard('~/notebooks/logs-gridsearch', write_graph=True, write_grads=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)]
    return callbacks

Next, we define an objective function that we want to minimize. This function includes the network architecture, hyperparameters, results, and everything else you want to include. Make sure you include the status STATUS_OK in your return statement for successful runs:

In [44]:
def f_nn(pars): 
    print ('Parameters: ', pars)
    xception_weights = 'pretrained-models/xception_weights_tf_dim_ordering_tf_kernels_notop.h5'
    x_input = Input(shape=img_dim, name="input")

    # model = Sequential()

    conv1 = (Xception(weights = xception_weights , include_top=False,pooling = 'avg'))(x_input)

    conv2 = (Dense(units=nb_classes, activation='softmax'))(conv1)
    
    model = Model(inputs=[x_input], outputs=[conv2])
    
    #################################################################################################
   
    model.compile(loss='categorical_crossentropy', optimizer=opt1, metrics=['accuracy'])
    
    #################################################################################################
    
    history = model.fit_generator(train_generator, epochs=pars['n_epochs'], steps_per_epoch= 300, 
              validation_data = validation_generator,
              validation_steps=100,
              verbose = 1, callbacks=get_callbacks(pars)) 
    
    best_epoch = np.argmax(history.history['val_acc'])
    best_val_acc = np.max(history.history['val_acc'])
    print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'val_acc': best_val_acc, 'best_epoch': best_epoch, 'status': STATUS_OK}

We can now initialize, start our grid search, and print the final results as follows:

In [ ]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=50, trials=trials)
print(best)

Parameters:  {'n_epochs': 100, 'optimizer': 'opt_adam'}
Epoch 1/100
300/300 [==============================] - 1797s 6s/step - loss: 0.0560 - acc: 0.9844 - val_loss: 0.0715 - val_acc: 0.9688
Epoch 2/100
300/300 [==============================] - 1786s 6s/step - loss: 0.0199 - acc: 0.9946 - val_loss: 0.2335 - val_acc: 0.9062
Epoch 3/100
300/300 [==============================] - 1791s 6s/step - loss: 0.0106 - acc: 0.9971 - val_loss: 0.0669 - val_acc: 0.9688
Epoch 4/100
300/300 [==============================] - 1789s 6s/step - loss: 9.3674e-04 - acc: 1.0000 - val_loss: 0.0483 - val_acc: 0.9688
Epoch 5/100
300/300 [==============================] - 1790s 6s/step - loss: 0.0117 - acc: 0.9964 - val_loss: 0.0167 - val_acc: 1.0000
Epoch 6/100
300/300 [==============================] - 1790s 6s/step - loss: 0.0094 - acc: 0.9986 - val_loss: 0.0238 - val_acc: 1.0000
Epoch 7/100
300/300 [==============================] - 1791s 6s/step - loss: 0.0013 - acc: 0.9997 - val_loss: 0.0164 - val_acc: 1.